In this notebook, I will experiment with different schedulers, seeing which can improve the standard coursework 1 setup.

In [ ]:
import numpy
import logging
from mlp.dataset import MNISTDataProvider

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info('Initialising data providers...')

train_dp = MNISTDataProvider(dset='train', batch_size=100, max_num_batches=1000, randomize=True)
valid_dp = MNISTDataProvider(dset='valid', batch_size=10000, max_num_batches=-10, randomize=False)
test_dp = MNISTDataProvider(dset='eval', batch_size=10000, max_num_batches=-10, randomize=False)

In [ ]:
#Baseline experiment

from mlp.layers import MLP, Linear, Sigmoid, Softmax #import required layer types
from mlp.optimisers import SGDOptimiser #import the optimiser

from mlp.costs import CECost #import the cost we want to use for optimisation
from mlp.schedulers import LearningRateExponential

logger = logging.getLogger()
logger.setLevel(logging.INFO)
rng = numpy.random.RandomState([2015,10,10])

#some hyper-parameters
nhid = 800
learning_rate = 0.5
max_epochs = 30
cost = CECost()
    
stats = []
for layer in xrange(1, 2):

    train_dp.reset()
    valid_dp.reset()
    test_dp.reset()
    
    #define the model
    model = MLP(cost=cost)
    model.add_layer(Sigmoid(idim=784, odim=nhid, irange=0.2, rng=rng))
    for i in xrange(1, layer):
        logger.info("Stacking hidden layer (%s)" % str(i+1))
        model.add_layer(Sigmoid(idim=nhid, odim=nhid, irange=0.2, rng=rng))
    model.add_layer(Softmax(idim=nhid, odim=10, rng=rng))

    # define the optimiser, here stochasitc gradient descent
    # with fixed learning rate and max_epochs
    # training_size should equal batch size, as that is the amount for each epoch
    lr_scheduler = LearningRateExponential(start_rate=learning_rate, max_epochs=max_epochs, training_size=100)
    optimiser = SGDOptimiser(lr_scheduler=lr_scheduler)

    logger.info('Training started...')
    tr_stats, valid_stats = optimiser.train(model, train_dp, valid_dp)

    logger.info('Testing the model on test set:')
    tst_cost, tst_accuracy = optimiser.validate(model, test_dp)
    logger.info('MNIST test set accuracy is %.2f %%, cost (%s) is %.3f'%(tst_accuracy*100., cost.get_name(), tst_cost))
    
    stats.append((tr_stats, valid_stats, (tst_cost, tst_accuracy)))

In [ ]:
#Baseline experiment

from mlp.layers import MLP, Linear, Sigmoid, Softmax #import required layer types
from mlp.optimisers import SGDOptimiser #import the optimiser

from mlp.costs import CECost #import the cost we want to use for optimisation
from mlp.schedulers import LearningRateNewBob

logger = logging.getLogger()
logger.setLevel(logging.INFO)
rng = numpy.random.RandomState([2015,10,10])

#some hyper-parameters
nhid = 800
learning_rate = 0.8
max_epochs = 30
cost = CECost()
    
stats = []
for layer in xrange(1, 2):

    train_dp.reset()
    valid_dp.reset()
    test_dp.reset()
    
    #define the model
    model = MLP(cost=cost)
    model.add_layer(Sigmoid(idim=784, odim=nhid, irange=0.2, rng=rng))
    for i in xrange(1, layer):
        logger.info("Stacking hidden layer (%s)" % str(i+1))
        model.add_layer(Sigmoid(idim=nhid, odim=nhid, irange=0.2, rng=rng))
    model.add_layer(Softmax(idim=nhid, odim=10, rng=rng))

    # define the optimiser, here stochasitc gradient descent
    # with fixed learning rate and max_epochs
    lr_scheduler = LearningRateNewBob(start_rate=learning_rate, max_epochs=max_epochs,\
                                      min_derror_stop=.05, scale_by=0.05, zero_rate=0.5, patience = 10)
    optimiser = SGDOptimiser(lr_scheduler=lr_scheduler)

    logger.info('Training started...')
    tr_stats, valid_stats = optimiser.train(model, train_dp, valid_dp)

    logger.info('Testing the model on test set:')
    tst_cost, tst_accuracy = optimiser.validate(model, test_dp)
    logger.info('MNIST test set accuracy is %.2f %%, cost (%s) is %.3f'%(tst_accuracy*100., cost.get_name(), tst_cost))
    
    stats.append((tr_stats, valid_stats, (tst_cost, tst_accuracy)))

In [1]:
#Baseline experiment
%autoreload
import numpy
import logging
from mlp.dataset import MNISTDataProvider

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info('Initialising data providers...')

train_dp = MNISTDataProvider(dset='train', batch_size=10, max_num_batches=100, randomize=True)
valid_dp = MNISTDataProvider(dset='valid', batch_size=10000, max_num_batches=-10, randomize=False)
test_dp = MNISTDataProvider(dset='eval', batch_size=10000, max_num_batches=-10, randomize=False)
from mlp.layers import MLP, Linear, Sigmoid, Softmax #import required layer types
from mlp.optimisers import SGDOptimiser #import the optimiser

from mlp.costs import CECost #import the cost we want to use for optimisation
from mlp.schedulers import LearningRateNewBob, LearningRateFixed

logger = logging.getLogger()
logger.setLevel(logging.INFO)
rng = numpy.random.RandomState([2015,10,10])

#some hyper-parameters
nhid = 600
learning_rate = 0.05
max_epochs = 10
cost = CECost()
    
stats = []
layer=2

train_dp.reset()

#define the model
model = MLP(cost=cost)
model.add_layer(Sigmoid(idim=784, odim=600, irange=0.2, rng=rng))
model.add_layer(Sigmoid(idim=600, odim=500, irange=0.2, rng=rng))
model.add_layer(Sigmoid(idim=500, odim=300, irange=0.2, rng=rng))
model.add_layer(Softmax(idim=300, odim=10, rng=rng))

lr_scheduler = LearningRateFixed(learning_rate=0.05, max_epochs=max_epochs)
optimiser = SGDOptimiser(lr_scheduler=lr_scheduler)

logger.info('Pre-Training started...')
tr_stats, valid_stats = optimiser.pretrain(model, train_dp, None)
logger.info('Training started...')

train_dp.reset()

tr_stats, valid_stats = optimiser.train(model, train_dp, valid_dp)

ERROR: Line magic function `%autoreload` not found.
INFO:root:Initialising data providers...
INFO:root:Pre-Training started...
INFO:mlp.optimisers:Max epochs 10
INFO:mlp.optimisers:epochs 0
INFO:mlp.optimisers:Running
INFO:mlp.optimisers:Epoch 1: Training cost (ce) is 21.023. Accuracy is 0.80%
INFO:mlp.optimisers:Epoch 2: Training cost (ce) is 11.791. Accuracy is 1.40%
INFO:mlp.optimisers:Epoch 3: Training cost (ce) is 8.838. Accuracy is 1.20%
INFO:mlp.optimisers:Epoch 4: Training cost (ce) is 7.072. Accuracy is 1.10%
INFO:mlp.optimisers:Epoch 5: Training cost (ce) is 5.876. Accuracy is 0.70%
INFO:mlp.optimisers:Epoch 6: Training cost (ce) is 5.021. Accuracy is 0.80%
INFO:mlp.optimisers:Epoch 7: Training cost (ce) is 4.384. Accuracy is 0.80%
INFO:mlp.optimisers:Epoch 8: Training cost (ce) is 3.893. Accuracy is 0.90%
INFO:mlp.optimisers:Epoch 9: Training cost (ce) is 3.502. Accuracy is 0.90%
INFO:mlp.optimisers:Epoch 10: Training cost (ce) is 3.182. Accuracy is 1.00%
INFO:mlp.optimisers

In [ ]:
import numpy
import logging
from mlp.dataset import MNISTDataProvider

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info('Initialising data providers...')

train_dp = MNISTDataProvider(dset='train', batch_size=10, max_num_batches=100, randomize=True)
i = 0
inputs=[]

for x,t in train_dp:
    inputs.append(x)
    
print inputs[0].shape
print len(inputs)

In [ ]:
print len(inputs)